
#  How compilers translate your program


Enter your name and student ID.

 * Name:
 * Student ID:



<a name="intro"> </a>
# 1. Introduction
* You look at assembly code generated by the compiler to get a good sense of how source programs are compiled into machine (assembly) code



# 2. Let the compiler generate assembly code
* you first have to master how to let your compiler emit assembly code 
* this is language- or compiler-dependent


## 2-1. Go
* ordinary go compiler does not support emitting native 64 bit x86 assembly
* it instead emits something called plan 9 assembly ([details](https://go.dev/doc/asm)) which abstracts away important details
* in this exercise, we instead use an alternative compiler, `gccgo`, which generates a native assembly for x86
* here is a minimum example

In [ ]:
%%writefile go/add123.go

package pl06
func Add123(n int64) int64 {
    return n + 123
}

In [ ]:
gccgo -O -g0 -S go/add123.go -o go/add123.s
cat go/add123.s


* find the label starting the function `Add123`, which may not be exactly `Add123`

* **NOTE 1:**
  * you need to say `package xxx`, with _xxx_ anything other than `main`; if you say `package main`, the compiler complains about the lack of `main` function
  * you need to <font color="red">capitalize the function name</font> whose assembly code you want to see, as otherwise the compiler may eliminate it as a dead code
  * in Go, functions starting with an uppercase letter are visible (can be called) from another package (see https://go.dev/doc/tutorial/create-module), so the compiler should leave the assembly code even if it finds the function not called anywhere in the file it is compiling

* **NOTE 2:** if you want to use `gccgo` along with `go build`, you can do
```
go build -compiler gccgo 
```
instead of just 
```
go build
```

## 2-2. Julia
* Julia has <font color="blue">code_native</font> function that prints assembly code of a function given particular concrete types for its input parameters
* here is a minimum example

In [ ]:
%%writefile jl/add123.jl

import InteractiveUtils

function add123(n)
    n + 123
end

InteractiveUtils.code_native(add123, (Int,))

In [ ]:
julia jl/add123.jl

* find the label starting the function `add123`, which may not be exactly `add123`

* note that the function `add123` does not declare the type of the input parameter (`n`); instead, the second parameter of `code_native`, `(Int,)` instructs Julia to compile `add123` given the type of the input is `Int`
* you can see how the same function is compiled for floating point numbers just by `code_native(add123, (Float64,))`


## 2-3. OCaml
* `ocamlopt` is the native compiler for OCaml, which is internally called by `dune`
* a minimum example source code

In [ ]:
%%writefile ml/add123.ml

let add123 n = n + 123
;;

In [ ]:
ocamlopt -c -S ml/add123.ml
cat ml/add123.s

* find the label starting the function `add123`, which may not be exactly `add123`

## 2-4. Rust
* `rustc` is a compiler for Rust, which is internally called by `cargo`
* here is a minimum example

In [ ]:
%%writefile rs/add123.rs

#[no_mangle]
pub fn add123(n : i64) -> i64 {
    n + 123
}

In [ ]:
rustc -O --emit asm --crate-type lib rs/add123.rs -o rs/add123.s
cat rs/add123.s

* try to find out where the function starts and instructions actually performing the addition

* **NOTE:**
  * `--crate-type lib` says you are building a library called from another program, not an executable; without it, the compiler complains about the lack of `main` function
  * #[no_mangle] guarantees the label name corresponding to `add123` is exactly `add123` and has a side-effect of preventing the compiler from eliminating the function as dead code


# 3. How to work on this exercise
* discuss with team members
* write code in the terminal, not in this notebook
* make your source code directly under one of `{ml,jl,go,rs}` directories using a text editor
* run `make` in the respective directory and it will compile your files for you



# <font color="green"> Problem 1 :  Examine calling convention</font>
* make a file `xx/ex01_many_args.xx` (xx is one of {ml, jl, go, rs})
* in it, define a function `many_args` (or `Many_args` in Go) that takes many (> 10) integer parameters and returns the sum of them (similar to the following C function) in your language, compile it into assembly and examine it

In [ ]:
%%writefile many_args.c

long many_args(long a00, long a01, long a02, long a03, long a04, long a05,
               long a06, long a07, long a08, long a09, long a10, long a11) {
  return a00 + a01 + a02 + a03 + a04 + a05 + a06 + a07 + a08 + a09 + a10 + a11;
}

* __<font color="blue">Questions:</font>__
  * how are parameters passed in each language? (which registers? stack?)
  * how does your language represent an integer $x$? is it represented in the same way as C?
  * does your language's calling convention differ from the C calling convention?



# <font color="green"> Problem 2 :  Examine how floating point numbers are represented</font>
* make a file `xx/ex02_add_floats.xx` (xx is one of {ml, jl, go, rs})
* in it, define a function `add_floats` (or `Add_floats` in Go) that takes two floating point numbers and returns the sum of the two (similar to the following C function) in your language, compile it into assembly and examine it

In [ ]:
%%writefile add_floats.c

double add_floats(double x, double y) {
  return x + y;
}

* __<font color="blue">Questions:</font>__
  * how are the two parameters passed in each language? (which registers? stack?)
  * how does your language represent a floating point number $x$? is it represented in the same way as C?
  * is there any language whose floating point number computations will be significantly slower than C?



# <font color="green"> Problem 3 :  Examine how arrays are represented</font>
* make a file `xx/ex03_get_elem_vector_floats.xx` (xx is one of {ml, jl, go, rs})
* in it, define a function `get_elem_vector_floats` (or `Get_elem_vector_floats` in Go) that takes an array of floating point numbers, `a`, and an integer, `i`, and returns `i`-th element of `a` (similar to to the following C++ function) in your language, compile it into assembly and examine it
* for an array, use
  * `[]float64` (slice) for Go
  * `Vector{Float64}` for Julia
  * `float array` (slice) for Go
  * `Vec<f64>` for Rust

In [ ]:
%%writefile get_elem_vector_floats.c

double get_float_array_elem_const(double a[10]) {
  return a[2];
}
double get_float_array_elem_i(double a[10], long i) {
  return a[i];
}

* __<font color="blue">Questions:</font>__
  * how does each language represent the respective array-like data structure?
  * how does each language detect out-of-bound array access? 



# <font color="green"> Problem 4 :  Examine how structs are represented</font>
* make a file `xx/ex04_get_struct_elem.xx` (xx is one of {ml, jl, go, rs})
* in it, define a struct `point` (or `Point` in Go) that has two fields (`x` and `y`; both are 64 bit floating point numbers) and function `get_struct_elem` (or `Get_struct_elem` in Go) that takes a point or a pointer to it and returns its `y` field

In [ ]:
%%writefile get_struct_elem.c

typedef struct {
  double x;
  double y;
} point_t;  
double get_point_y(point_t p) {
  return p.y;
}
double get_pointp_y(point_t * p) {
  return p->y;
}

* __<font color="blue">Questions:</font>__
  * how does each language represent multiword structures?
  * how does each language detect out-of-bound array access? 



# <font color="green"> Problem 5 :  Examine how arrays are scanned</font>
* make a file `xx/ex05_sum_vector_floats.xx` (xx is one of {ml, jl, go, rs})
* in it, define a function `sum_vector_floats` (or `Sum_vector_floats` in Go) that takes an array of floating point numbers and returns the sum of all its elements (similar to to the following C++ function) in your language, compile it into assembly and examine it
* use
  * `[]float64` (slice) for Go
  * `Vector{Float64}` for Julia
  * `float array` (slice) for Go
  * `Vec<f64>` for Rust

In [ ]:
%%writefile sum_vector_floats.cc

double sum_array_floats(double * a, long n) {
  double s = 0.0;
  for (int i = 0; i < n; i++) {
    s += a[i];
  }
  return s;
}

* __<font color="blue">Questions:</font>__
  * given that the array has a large number of elements, identify the loop that takes most of the time
  * in that loop, how many instructions are executed per array element?
  * what does each language/compiler do to optimize it?
